In [ ]:
# 定义氨基酸
Amino_acids = ['ALA', 'CYS', 'ASP', 'GLU', 'PHE', 'GLY', 'HIS', 'ILE', 'LYS', 'LEU', 'MET', 'ASN', 'PRO', 'GLN', 'ARG', 'SER', 'THR', 'VAL', 'TRP', 'TYR']
# 定义缩写氨基酸
amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
Amino_acids_num = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,20]
Amino_acids_dict = {}
for i in range(len(Amino_acids)):
    Amino_acids_dict[amino_acids[i]] = Amino_acids_num[i]

def protein_to_int(protein):
    protein_int = []
    for i in range(len(protein)):
        temp = protein[i]

        index = [i for i, x in enumerate(amino_acids) if x == temp]
        if len(index) != 0:
            protein_int.append(index[0])
    return protein_int



In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import torch

dir = 'data/data.csv'
df = pd.read_csv(dir)
seq = df['Sequence']
label = df['Label']
print(len(label))
input_seq = []
for i in seq:

    temp = protein_to_int(i)

    input_seq.append(torch.tensor(temp))
input_seq = pad_sequence(input_seq,padding_value=-1)
print(input_seq.shape)

input_seq= torch.transpose(input_seq,1,0)
input_label = torch.tensor(label)



4851
torch.Size([149, 4851])


In [13]:
print(input_seq.shape)
print(input_label.shape)

torch.Size([4851, 149])
torch.Size([4851])


In [23]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split   # 数据集划分
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV  # 带交叉验证的网格搜索
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


import pandas as pd
from time import time
import datetime
import pickle

from sklearn.metrics import accuracy_score, mean_absolute_error, f1_score, recall_score


In [3]:
from evaluate_metrics import *

In [ ]:
from sklearn import svm
from sklearn.model_selection import KFold,StratifiedKFold
sfolder = StratifiedKFold(n_splits=5,random_state=2,shuffle=True)
result = np.random.randn(5,5)
for i,(train_id,test_id) in enumerate(sfolder.split(input_seq,input_label)):

    Xtrain,Ytrain = input_seq[train_id],input_label[train_id]
    Xtest,Ytest = input_seq[test_id],input_label[test_id]

    clf =svm.SVC(gamma=0.05,max_iter=200,decision_function_shape='ovo')

    clf.fit(Xtrain,Ytrain)
    print('训练集',clf.score(Xtrain,Ytrain))
    print('测试集',clf.score(Xtest,Ytest))
    tra_label = clf.predict(Xtrain)
    tes_label = clf.predict(Xtest)
    from sklearn.metrics import precision_score
    val_result = [accuracy_score(Ytest, tes_label),precision_score(Ytest,tes_label,average='weighted'),
                  recall_score(Ytest, tes_label), f1_score(Ytest, tes_label),
              mcc_score(Ytest, tes_label)]
    for j in range(5):
        result[j][i] = val_result[j]

for i in range(5):
    print(np.mean(result[i]))

In [36]:
from sklearn.ensemble import RandomForestClassifier
sfolder = StratifiedKFold(n_splits=5,random_state=2,shuffle=True)
result = np.random.randn(5,5)
for i,(train_id,test_id) in enumerate(sfolder.split(input_seq,input_label)):

    glassRFModel = RandomForestClassifier(n_estimators=100,max_depth=None,random_state=2)
    glassRFModel.fit(Xtrain,Ytrain)
    tes_label = glassRFModel.predict(Xtest)


    val_result = [accuracy_score(Ytest, tes_label),precision_score(Ytest,tes_label,average='weighted'),
                  recall_score(Ytest, tes_label), f1_score(Ytest, tes_label),
              mcc_score(Ytest, tes_label)]
    for j in range(5):
        result[j][i] = val_result[j]

for i in range(5):
    print(np.mean(result[i]))


0.956701030927835
0.9567660464597477
0.956701030927835
0.9567291438816673
0.905540035549784


In [44]:
import xgboost as xgb

sfolder = StratifiedKFold(n_splits=5,random_state=2,shuffle=True)
result = np.random.randn(5,5)
for i,(train_id,test_id) in enumerate(sfolder.split(input_seq,input_label)):
    xgb_clf = xgb.XGBClassifier(
    learning_rate = 0.1,
    max_depth = 100,
    n_estimators =100,
    objective = 'multi:softmax',
    num_class = 4,
    random_state = 42
)
    xgb_clf.fit(Xtrain,Ytrain)
    tes_label = xgb_clf.predict(Xtest)
    val_result = [accuracy_score(Ytest, tes_label),precision_score(Ytest,tes_label,average='weighted'),
                  recall_score(Ytest, tes_label), f1_score(Ytest, tes_label),
              auc_score(Ytest, tes_label)]
    for j in range(5):
        result[j][i] = val_result[j]
    print(val_result)
for i in range(5):
    print(np.mean(result[i]))

[0.9556701030927836, 0.9555772560188303, 0.9556701030927836, 0.9555960074549301, 0.9499405602199271]
[0.9556701030927836, 0.9555772560188303, 0.9556701030927836, 0.9555960074549301, 0.9499405602199271]
[0.9556701030927836, 0.9555772560188303, 0.9556701030927836, 0.9555960074549301, 0.9499405602199271]
[0.9556701030927836, 0.9555772560188303, 0.9556701030927836, 0.9555960074549301, 0.9499405602199271]
[0.9556701030927836, 0.9555772560188303, 0.9556701030927836, 0.9555960074549301, 0.9499405602199271]
0.9556701030927834
0.9555772560188303
0.9556701030927834
0.9555960074549301
0.9499405602199271


In [45]:
from sklearn.tree import DecisionTreeClassifier
sfolder = StratifiedKFold(n_splits=5,random_state=2,shuffle=True)
result = np.random.randn(5,5)
for i,(train_id,test_id) in enumerate(sfolder.split(input_seq,input_label)):
    tree_clf = DecisionTreeClassifier()
    tree_clf.fit(Xtrain,Ytrain)
    tes_label = tree_clf.predict(Xtest)
    val_result = [accuracy_score(Ytest, tes_label),precision_score(Ytest,tes_label,average='weighted'),
                  recall_score(Ytest, tes_label), f1_score(Ytest, tes_label),
              auc_score(Ytest, tes_label)]
    print(val_result)
    for j in range(5):
        result[j][i] = val_result[j]
for i in range(5):
    print(np.mean(result[i]))

[0.9175257731958762, 0.9175257731958762, 0.9175257731958762, 0.9175257731958762, 0.9099115833271417]
[0.9144329896907216, 0.9140803491270008, 0.9144329896907216, 0.914171257587136, 0.9035868192287688]
[0.9175257731958762, 0.9175257731958762, 0.9175257731958762, 0.9175257731958762, 0.9099115833271417]
[0.9195876288659793, 0.919407240559454, 0.9195876288659793, 0.9194805501507396, 0.910854261089234]
[0.9195876288659793, 0.9195876288659793, 0.9195876288659793, 0.9195876288659793, 0.9121637937439632]
0.9177319587628865
0.9176253529888374
0.9177319587628865
0.9176581965991215
0.9092856081432499
